In [1]:
import getpass
import os
from dotenv import load_dotenv
from langchain_community.graphs import Neo4jGraph
import pandas as pd
from langchain.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI

In [2]:
load_dotenv() # Loads OpenAI API Key and Neo4J URI, Username and Password from .env file

True

In [6]:
graph = Neo4jGraph()

data_query = """
LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/msakthiganesh/Langchain-Pandas-Dataframe-Agent-using-Local-Model/main/take_home_dataset_cleaned.csv' AS row
MERGE (o:Order {id:row.Order_ID})
SET o.order_date = date(row.Order_date),
    o.start_shipping_date = date(row.Start_Shipping_Date),
    o.estimated_arrival_date = date(row.Estimated_Arrival_Date),
    o.actual_arrival_date = date(row.Actual_Arrival_Date),
    o.days_from_order_to_shipment = toInteger(row.Days_from_order_to_shipment),
    o.days_from_shipment_to_delivery = toInteger(row.Days_from_shipment_to_delivery),
    o.days_from_order_to_delivery = toInteger(row.Days_from_order_to_delivery),
    o.days_between_estimated_and_actual_arrival = toInteger(row.Days_between_estimated_and_actual_arrival),
    o.status = row.Status
MERGE (p:ProductCategory {name:row.Product_Category})
MERGE (m:ModeOfTransport {name:row.Mode_Of_Transport})
MERGE (pr:Priority {name:row.Priority})
MERGE (c:Carrier {name:row.Carrier_name})
MERGE (w:Warehouse {name:row.Warehouse})
MERGE (s:Supplier {name:row.Supplier_Name})
MERGE (cu:Customer {name:row.Customer_Name})
MERGE (d:DeliveryDistance {value:toInteger(row.Delivery_distance)})
MERGE (o)-[:IN_CATEGORY]->(p)
MERGE (o)-[:USES_TRANSPORT]->(m)
MERGE (o)-[:HAS_PRIORITY]->(pr)
MERGE (o)-[:SHIPPED_BY]->(c)
MERGE (o)-[:FROM_WAREHOUSE]->(w)
MERGE (o)-[:FROM_SUPPLIER]->(s)
MERGE (o)-[:TO_CUSTOMER]->(cu)
MERGE (o)-[:DELIVERY_DISTANCE]->(d)

"""

graph.query(data_query)

[]

In [7]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Order {id: STRING, order_date: DATE, start_shipping_date: DATE, estimated_arrival_date: DATE, actual_arrival_date: DATE, days_from_order_to_shipment: INTEGER, days_from_shipment_to_delivery: INTEGER, days_from_order_to_delivery: INTEGER, days_between_estimated_and_actual_arrival: INTEGER, status: STRING}
ProductCategory {name: STRING}
ModeOfTransport {name: STRING}
Priority {name: STRING}
Carrier {name: STRING}
Warehouse {name: STRING}
Supplier {name: STRING}
Customer {name: STRING}
DeliveryDistance {value: INTEGER}
Relationship properties:

The relationships:
(:Order)-[:IN_CATEGORY]->(:ProductCategory)
(:Order)-[:USES_TRANSPORT]->(:ModeOfTransport)
(:Order)-[:HAS_PRIORITY]->(:Priority)
(:Order)-[:SHIPPED_BY]->(:Carrier)
(:Order)-[:FROM_WAREHOUSE]->(:Warehouse)
(:Order)-[:FROM_SUPPLIER]->(:Supplier)
(:Order)-[:TO_CUSTOMER]->(:Customer)
(:Order)-[:DELIVERY_DISTANCE]->(:DeliveryDistance)


In [8]:
llm = ChatOpenAI(model="gpt-4", temperature=0)
chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True)
response = chain.invoke({"query": "What is the maximum delivery distance?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (o:Order)-[:DELIVERY_DISTANCE]->(d:DeliveryDistance)
RETURN MAX(d.value)
Full Context:
[{'MAX(d.value)': 9900}]

> Finished chain.


{'query': 'What is the maximum delivery distance?',
 'result': 'The maximum delivery distance is 9900.'}

In [9]:
chain = GraphCypherQAChain.from_llm(
    graph=graph, llm=llm, verbose=True, validate_cypher=True
)
response = chain.invoke({"query": "What are the modes of transport?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (o:Order)-[:USES_TRANSPORT]->(m:ModeOfTransport) RETURN DISTINCT m.name
Full Context:
[{'m.name': 'Less Than Truckload'}, {'m.name': 'Full Truckload'}, {'m.name': 'Air Freight'}, {'m.name': 'Intermodal & Rail Shipments'}]

> Finished chain.


{'query': 'What are the modes of transport?',
 'result': 'The modes of transport are Less Than Truckload, Full Truckload, Air Freight, and Intermodal & Rail Shipments.'}